In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2 as pg
import ipywidgets as wg
import warnings
from dotenv import load_dotenv
import os

warnings.filterwarnings('ignore')


load_dotenv()


DB_USER_1 = os.getenv('DB_USER_1')
DB_PASSWORD_1 = os.getenv('DB_PASSWORD_1')
DB_PORT_1 = os.getenv('DB_PORT_1')
DB_HOST_1 = os.getenv('DB_HOST_1')
DB_NAME_2 = os.getenv('DB_NAME_2')

# Connect to the database
conn = pg.connect(
    host=DB_HOST_1,
    database=DB_NAME_2,
    user=DB_USER_1,
    password=DB_PASSWORD_1,
    port=DB_PORT_1
)

#A)
with conn.cursor() as cursor:
    cursor.execute("WITH CitasPorHorario AS ( "
    "SELECT h.hora_inicio, c.nombre_comuna, s.nombre_sede, s.id_sede, COUNT(fc.id_cita) AS total_citas "
    "FROM fact_cita fc "
    "JOIN horario h ON fc.id_horario = h.id_horario "
    "JOIN comuna c ON fc.id_comuna = c.id_comuna "
    "JOIN sede s ON fc.id_sede = s.id_sede "
    "GROUP BY "
    "h.hora_inicio, c.nombre_comuna, s.nombre_sede, s.id_sede"
"), MaxCitasPorSede AS ( "
    "SELECT id_sede, MAX(total_citas) AS max_citas "
    "FROM CitasPorHorario "
    "GROUP BY id_sede "
") "
"SELECT ch.hora_inicio,ch.nombre_comuna,ch.nombre_sede,ch.total_citas "
"FROM CitasPorHorario ch "
"JOIN MaxCitasPorSede mcs ON ch.id_sede = mcs.id_sede AND ch.total_citas = mcs.max_citas; ")
    data_a = cursor.fetchall()

#B)
with conn.cursor() as cursor:
    cursor.execute("WITH GastoPorCliente AS ( "
    "SELECT c.id_cliente,c.nombre_cliente,c.apellido_cliente,cl.nombre_comuna AS comuna_cliente, "
    "s.nombre_sede,se.nombre_comuna AS comuna_sede,SUM(sv.precio_servicio) AS total_gasto "
    "FROM fact_cita fc "
    "JOIN cliente c ON fc.id_cliente = c.id_cliente "
    "JOIN comuna cl ON c.comuna = cl.nombre_comuna "
    "JOIN sede s ON fc.id_sede = s.id_sede "
    "JOIN comuna se ON s.direccion_sede = se.nombre_comuna "
    "JOIN servicio sv ON fc.id_servicio = sv.id_servicio "
    "GROUP BY c.id_cliente, c.nombre_cliente, c.apellido_cliente, cl.nombre_comuna, s.nombre_sede, se.nombre_comuna "
"), RankedGastoPorSede AS ( "
    "SELECT gc.*,ROW_NUMBER() OVER (PARTITION BY gc.nombre_sede ORDER BY gc.total_gasto DESC) AS rn "
    "FROM GastoPorCliente gc "
") "
"SELECT nombre_cliente,apellido_cliente,comuna_cliente,nombre_sede,comuna_sede,total_gasto "
"FROM RankedGastoPorSede "
"WHERE rn = 1; "
)
    data_b = cursor.fetchall()

#C)
with conn.cursor() as cursor:
    cursor.execute("WITH IngresosPorPeluquero AS ( "
    "SELECT e.id_empleado,e.nombre AS nombre_empleado,e.apellido AS apellido_empleado,s.nombre_sede, "
        "EXTRACT (MONTH FROM fc.fecha) AS mes,ROUND(SUM(sv.precio_servicio + COALESCE(e.comision, 0))::numeric, 2) AS total_ingresos "
    "FROM fact_cita fc "
    "JOIN empleado e ON fc.id_empleado = e.id_empleado "
    "JOIN sede s ON fc.id_sede = s.id_sede "
    "JOIN servicio sv ON fc.id_servicio = sv.id_servicio "
    "WHERE fc.fecha BETWEEN '2023-01-01' AND '2023-12-31' "
    "GROUP BY e.id_empleado, e.nombre, e.apellido, s.nombre_sede, EXTRACT (MONTH FROM fc.fecha) "
"), RankedIngresosPorMes AS ( "
    "SELECT ip.*,ROW_NUMBER() OVER (PARTITION BY ip.mes, ip.nombre_sede ORDER BY ip.total_ingresos DESC) AS rn "
    "FROM IngresosPorPeluquero ip "
") "
"SELECT nombre_empleado,apellido_empleado,nombre_sede,mes,total_ingresos "
"FROM RankedIngresosPorMes "
"WHERE rn = 1 "
"ORDER BY mes, nombre_sede; "
)
    data_c = cursor.fetchall()

#D)
with conn.cursor() as cursor:
    cursor.execute("SELECT c.nombre_cliente, c.apellido_cliente, sexo, se.nombre "
"FROM cliente c "
"INNER JOIN fact_cita fc ON c.id_cliente = fc.id_cliente "
"INNER JOIN servicio se ON fc.id_servicio = se.id_servicio "
"WHERE c.sexo = 'Masculino' AND (se.id_servicio = 1 OR se.id_servicio = 10);"
)
    data_d = cursor.fetchall()

#E)
with conn.cursor() as cursor:
    cursor.execute( "SELECT "
    "cl.nombre_cliente || ' ' || cl.apellido_cliente AS nombre_cliente, "
    "cl.comuna AS comuna_cliente, "
    "s.nombre_sede AS peluqueria, "
    "p.monto AS valor_pagado "
"FROM "
    "fact_cita fc "
"JOIN "
    "cliente cl ON fc.id_cliente = cl.id_cliente "
"JOIN "
    "sede s ON fc.id_sede = s.id_sede "
"JOIN "
    "pago p ON fc.id_pago = p.id_pago "
"WHERE "
    "fc.id_servicio = 6; "
)
    data_e = cursor.fetchall()

#F)
with conn.cursor() as cursor:
    cursor.execute("WITH HorariosConcurridos AS ( "
    "SELECT  "
        "s.nombre_sede, "
        "EXTRACT(YEAR FROM fc.fecha) AS anio, "
        "EXTRACT(MONTH FROM fc.fecha) AS mes, "
        "h.hora_inicio, "
        "COUNT(fc.id_cita) AS total_citas, "
        "ROW_NUMBER() OVER(PARTITION BY s.nombre_sede, EXTRACT(YEAR FROM fc.fecha), EXTRACT(MONTH FROM fc.fecha) ORDER BY COUNT(fc.id_cita) DESC) AS rn "
    "FROM  "
        "fact_cita fc "
    "JOIN "
        "sede s ON fc.id_sede = s.id_sede "
    "JOIN "
        "horario h ON fc.id_horario = h.id_horario "
    "WHERE "
        "EXTRACT(YEAR FROM fc.fecha) IN (2019, 2020) "
   " GROUP BY "
        "s.nombre_sede, EXTRACT(YEAR FROM fc.fecha), EXTRACT(MONTH FROM fc.fecha), h.hora_inicio "
") "
"SELECT "
    "nombre_sede, "
    "anio, "
   " mes,"
    "hora_inicio, "
    "total_citas "
"FROM "
    "HorariosConcurridos "
"WHERE " 
    "rn = 1 "
"ORDER BY "
    "nombre_sede, anio, mes; "
) 
    data_f = cursor.fetchall() 
    
#G) 
with conn.cursor() as cursor:
    cursor.execute("WITH DuracionCitas AS ( "
    "SELECT cl.nombre_cliente || ' ' || cl.apellido_cliente AS nombre_cliente, "
            "s.nombre_sede,EXTRACT(MONTH FROM fc.fecha) AS mes,SUM(srv.duracion) AS duracion_total_minutos, "
            "ROW_NUMBER() OVER(PARTITION BY s.nombre_sede, EXTRACT(MONTH FROM fc.fecha) ORDER BY SUM(srv.duracion) DESC) AS rn "
    "FROM fact_cita fc "
    "JOIN cliente cl ON fc.id_cliente = cl.id_cliente "
    "JOIN sede s ON fc.id_sede = s.id_sede "
    "JOIN servicio srv ON fc.id_servicio = srv.id_servicio "
    "GROUP BY cl.nombre_cliente, cl.apellido_cliente, s.nombre_sede, EXTRACT(MONTH FROM fc.fecha) "
")"
"SELECT nombre_cliente,nombre_sede,mes,duracion_total_minutos "
"FROM DuracionCitas "
"WHERE rn = 1 "
"ORDER BY nombre_sede, mes; "
    )
    data_g = cursor.fetchall()
   
#H) 
with conn.cursor() as cursor:
    cursor.execute("WITH ServicioMasCaroPorSede AS ( "
    "SELECT s.nombre_sede,srv.nombre AS nombre_servicio,srv.precio_servicio, "
            "ROW_NUMBER() OVER(PARTITION BY s.nombre_sede ORDER BY srv.precio_servicio DESC) AS rn "
    "FROM servicio srv "
    "JOIN fact_cita fc ON srv.id_servicio = fc.id_servicio "
    "JOIN sede s ON fc.id_sede = s.id_sede "
    ") "
    "SELECT nombre_sede,nombre_servicio,precio_servicio "
    "FROM ServicioMasCaroPorSede "
    "WHERE rn = 1 "
    "ORDER BY nombre_sede; "
    )
    data_h = cursor.fetchall()

#I)
with conn.cursor() as cursor:
    cursor.execute("WITH HorasTrabajadas AS ( "
    "SELECT e.nombre || ' ' || e.apellido AS nombre_peluquero, "
    "s.nombre_sede,EXTRACT(MONTH FROM fc.fecha) AS mes, "
    "SUM(EXTRACT(EPOCH FROM (h.hora_fin - h.hora_inicio)) / 3600) AS horas_trabajadas, "
    "ROW_NUMBER() OVER(PARTITION BY EXTRACT(MONTH FROM fc.fecha) ORDER BY SUM(EXTRACT(EPOCH FROM (h.hora_fin - h.hora_inicio)) / 3600) DESC) AS rn "
    "FROM fact_cita fc "
    "JOIN empleado e ON fc.id_empleado = e.id_empleado "
    "JOIN horario h ON fc.id_horario = h.id_horario "
    "JOIN sede s ON fc.id_sede = s.id_sede "
    "WHERE EXTRACT(YEAR FROM fc.fecha) = 2019 "
    "GROUP BY e.id_empleado, e.nombre, e.apellido, s.nombre_sede, EXTRACT(MONTH FROM fc.fecha) "
    ") "
    "SELECT "
    "nombre_peluquero,nombre_sede,mes,horas_trabajadas "
    "FROM HorasTrabajadas "
    "WHERE rn = 1 "
    "ORDER BY mes; "
    )
    data_i = cursor.fetchall()
    
#J)
with conn.cursor() as cursor:
    cursor.execute("SELECT c.nombre_comuna, COUNT(distinct(sede.nombre_sede)) AS cantidad_peluquerias, COUNT(distinct (cl.id_cliente)) AS cantidad_clientes "
    "FROM comuna c "
    "INNER JOIN fact_cita fc ON c.id_comuna = fc.id_comuna "
    "INNER JOIN sede ON fc.id_sede = sede.id_sede "
    "INNER JOIN cliente cl ON fc.id_cliente = cl.id_cliente "
    "GROUP BY c.nombre_comuna; "
    )
    data_j = cursor.fetchall()
        
        
        
    
    


In [ ]:
data_a = pd.DataFrame(data_a, columns=['hora_inicio', 'nombre_comuna', 'nombre_sede', 'total_citas'])


data_a["hora_inicio"] = pd.to_datetime(data_a["hora_inicio"], format='%H:%M:%S').dt.hour
data_a["sedexhora"] = data_a["hora_inicio"].astype(str)  + "hrs " + data_a["nombre_sede"]

x = data_a['sedexhora']
y = data_a['total_citas']

graf_a = plt.figure()
plt.barh(x, y)
plt.title('Hora con mayor cantidad de citas por sede')
plt.xlabel('Cantidad de citas')
plt.ylabel('Sede')

In [ ]:
data_b = pd.DataFrame(data_b, columns=['nombre_cliente', 'apellido_cliente', 'comuna_cliente', 'nombre_sede', 'comuna_sede', 'total_gasto'])

x = data_b['nombre_cliente'] + " " + data_b['apellido_cliente'] + " (" + data_b['nombre_sede'] + ")"
y = data_b['total_gasto']

graf_b = plt.figure()
plt.barh(x, y)
plt.title('Cliente con mayor gasto por sede')
plt.xlabel('Total gastado')
plt.ylabel('Cliente')


In [ ]:
data_c = pd.DataFrame(data_c, columns=['nombre_empleado', 'apellido_empleado', 'nombre_sede', 'mes', 'total_ingresos'])

dropdown = wg.Dropdown(
    options=list(data_c['nombre_sede'].unique()),
    description='Sede:',
    disabled=False
)

def graf_c(sede):
    data_c_f = data_c[data_c['nombre_sede'] == sede]
    data_c_f["peluqueroxmes"] = data_c_f["nombre_empleado"] + " " + data_c_f["apellido_empleado"] + " (" + data_c_f["mes"].astype(str) + ")"
    
    x = data_c_f['peluqueroxmes']
    y = data_c_f['total_ingresos']
    
    plt.figure(figsize=(10, 6))
    plt.barh(x, y)
    plt.title('Empleado con mayor ingreso en' + sede)
    plt.xlabel('Total ingresado')
    plt.ylabel('Empleado')
    plt.show()

wg.interactive(graf_c, sede=dropdown)


In [ ]:
data_d = pd.DataFrame(data_d, columns=['nombre_cliente', 'apellido_cliente', 'sexo', 'nombre_servicio'])
print("tabla de clientes hombres que han solicitado los servicios de corte de cabello y barba:")
data_d

In [ ]:
date_e = pd.DataFrame(data_e, columns=['nombre_cliente', 'comuna_cliente', 'peluqueria', 'valor_pagado'])
print("Lista de clientes que tiñen el pelo, indicando la comuna del cliente, la peluquería donde se atendió y el valor que pagó")
date_e


In [ ]:
data_f = pd.DataFrame(data_f, columns=['nombre_sede', 'anio', 'mes', 'hora_inicio', 'total_citas'])


dropdown = wg.Dropdown(
    options=list(data_f['nombre_sede'].unique()),
    description='Sede:',
    disabled=False
)

dropdown1 = wg.Dropdown(
    options=list(data_f['anio'].unique()),
    description='Año:',
    disabled=False
)

def graf_f(sede, anio):
    data_x = data_f[data_f['nombre_sede'] == sede]
    data_x = data_x[data_x['anio'] == anio]
    data_x["sedexhoraxmes"] = data_x["hora_inicio"].astype(str)  + "hrs " + data_x["nombre_sede"] + " (" + data_x["mes"].astype(str) + ")"
    
    y = data_x['total_citas']
    x = data_x['sedexhoraxmes']
    
    plt.figure(figsize=(10, 6))
    plt.barh(x, y)
    plt.title('Hora con mayor cantidad de citas en ' + sede)
    plt.xlabel('Cantidad de citas')
    plt.ylabel('Empleado')
    plt.show()

wg.interactive(graf_f, sede=dropdown, anio=dropdown1)

In [ ]:
dato_g = pd.DataFrame(data_g, columns=['nombre_cliente', 'nombre_sede', 'mes', 'duracion_total_minutos'])


dropdown = wg.Dropdown(
    options=list(dato_g['nombre_sede'].unique()),
    description='Sede:',
    disabled=False
)

def graf_g(sede):
    data_g_f = dato_g[dato_g['nombre_sede'] == sede]
    data_g_f["clientexmes"] = data_g_f["nombre_cliente"] + " (" + data_g_f["mes"].astype(str) + ")"
    
    x = data_g_f['clientexmes']
    y = data_g_f['duracion_total_minutos']
    
    plt.figure(figsize=(10, 6))
    plt.barh(x, y)
    plt.title('Cliente con mayor duración de servicios por sede')
    plt.xlabel('Total duración')
    plt.ylabel('Cliente')
    plt.show()
    
    
wg.interactive(graf_g, sede=dropdown)


In [ ]:
data_h = pd.DataFrame(data_h, columns=['nombre_sede', 'nombre_servicio', 'precio_servicio'])
data_h

In [ ]:
data_i = pd.DataFrame(data_i, columns=['nombre_peluquero', 'nombre_sede', 'mes', 'horas_trabajadas'])


x = data_i['nombre_peluquero'] + " (" + data_i['mes'].astype(str) + ")"
y = data_i['horas_trabajadas']

graf_i = plt.figure()
plt.barh(x, y)
plt.title('Peluquero con mayor cantidad de horas trabajadas por mes')
plt.xlabel('Total horas trabajadas')
plt.ylabel('Peluquero')


In [ ]:
data_j = pd.DataFrame(data_j, columns=['nombre_comuna', 'cantidad_peluquerias', 'cantidad_clientes'])

data_j